# Setup
## Constants

In [2]:
!pip install transformers -q
!pip install torch -q
!pip install torchvision -q

In [3]:
# False if you have already created and saved a .pth file to PTH_SAVE_PATH
CREATE_NEW_DATASET = True

# train, test, val set size. Should sum to 1
SET_SIZES = {
    "train": 0.8,
    "test": 0.1,
    "val": 0.1,
}

# samples per class in uniform dataset
N_SAMPLES = 400

# path to dataset (do not change)
HM_DATA_PATH = "../dataset/"

# path to pth saves (do not change)
PTH_SAVE_PATH = "../pth/"

## Imports

In [4]:
import os, sys, random, importlib, transformers, itertools, copy
import numpy as np, torch.nn as nn, torch, seaborn as sns, matplotlib.pyplot as plt, pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import DataLoader
print(os.getcwd())# Our own files
# sys.path.append('./src/')
import model_functions, utils, training, datasets
def set_seed(seed):# reproducable
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(0)

/opt/homebrew/Caskroom/miniconda/base/envs/ds-proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/daghjelm/Documents/kth/ds-proj/DD2430_Project/src


In [5]:
def update():# if you change our files
    import model_functions, utils, training, datasets
    for lib in [model_functions, utils, training, datasets]:
        importlib.reload(lib)# issues with not updating
update()

In [6]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available(): # For apple silicon
    device = 'mps'
print("Using device:", device)

Using device: mps


In [7]:
model = transformers.CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = transformers.CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
processor.feature_extractor.do_rescale = False # make sure image values: False=> [0-1] and True=> [0,255]

/opt/homebrew/Caskroom/miniconda/base/envs/ds-proj/lib/python3.12/site-packages/transformers/models/clip/processing_clip.py:149: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


In [21]:
print(type(processor))

<class 'transformers.models.clip.processing_clip.CLIPProcessor'>


### Load dataset and split

## Create new imbalanced dataset

Main class for dataset

In [9]:
main_class = 'garment_group_name'

### Full dataset, run once

In [10]:
df = pd.read_csv(HM_DATA_PATH+'articles_filtered.csv')
embs = torch.load(HM_DATA_PATH+'embedds.pth', weights_only=True) # all 100k embeddings
labs = torch.load(HM_DATA_PATH+'labels.pth', weights_only=True).tolist() #  100k labels
hmd = datasets.HMDatasetDuplicates(embs, np.array(labs), df)
print(hmd.article_id2suclass(694805002, main_class))
#print(hmd.list_article_id2suclass(labs, 'garment_group_name'))#all
print(len(labs))
BALANCED = False

Knitwear
105099


In [22]:
print(type(processor.tokenizer))

<class 'transformers.models.clip.tokenization_clip_fast.CLIPTokenizerFast'>


In [13]:
update()
batch_size = 32
set_sizes = {"train": 0.8, "val": 0.1}
data = datasets.datasets(embs, np.array(labs), df, set_sizes, True)# takes 3 min

47071 Train size: 37656, Val size: 4707, Test size: 4708
105099
This should be empty set() set()
The resulting sizes 84095 4707 4708


### Subsets

In [ ]:
update()
exclude_classes = ['Special Offers', 'Woven/Jersey/Knitted mix Baby','Unknown']
dataloaders_imbalanced = datasets.get_dataloaders('garment_group_name', data, 5000, exclude_classes, 32)# look at Resource Utilization to see if capping

100%|██████████| 84095/84095 [00:09<00:00, 9303.20it/s]


Final class count for train: {'Shoes': 3989, 'Jersey Basic': 5000, 'Swimwear': 2301, 'Dresses Ladies': 3947, 'Blouses': 4701, 'Dressed': 772, 'Trousers': 5000, 'Under-, Nightwear': 5000, 'Shirts': 1623, 'Trousers Denim': 2532, 'Knitwear': 5000, 'Dresses/Skirts girls': 1184, 'Shorts': 1228, 'Jersey Fancy': 5000, 'Accessories': 5000, 'Outdoor': 3644, 'Skirts': 1016, 'Socks and Tights': 1798}


100%|██████████| 4707/4707 [00:00<00:00, 9303.44it/s]


Final class count for val: {'Shoes': 267, 'Jersey Basic': 187, 'Swimwear': 115, 'Dresses Ladies': 244, 'Blouses': 277, 'Dressed': 49, 'Trousers': 261, 'Under-, Nightwear': 261, 'Shirts': 73, 'Trousers Denim': 138, 'Knitwear': 348, 'Dresses/Skirts girls': 99, 'Shorts': 55, 'Jersey Fancy': 832, 'Accessories': 685, 'Outdoor': 232, 'Skirts': 51, 'Socks and Tights': 65}


100%|██████████| 4708/4708 [00:00<00:00, 8544.05it/s]


Final class count for test: {'Shoes': 277, 'Jersey Basic': 200, 'Swimwear': 101, 'Dresses Ladies': 246, 'Blouses': 301, 'Dressed': 41, 'Trousers': 306, 'Under-, Nightwear': 249, 'Shirts': 77, 'Trousers Denim': 120, 'Knitwear': 315, 'Dresses/Skirts girls': 90, 'Shorts': 54, 'Jersey Fancy': 834, 'Accessories': 647, 'Outdoor': 254, 'Skirts': 73, 'Socks and Tights': 55}


In [20]:
dataloaders = dataloaders_imbalanced

## Add description to labels



In [ ]:
update()
BALANCED = False
def baseline(dataloaders, add):
    clip = {'m':model, 'p':processor}
    ft = training.FinetuneCLIP(dataloaders, clip)
    ft.conf = {'epochs': 400, 'balanced':BALANCED}
    ft.tt['soft'], ft.tt['LoRA'], ft.tt['desc'] = 0, 0, 1 #baseline
    ft.initialize({'add':add}, False)
    all_predictions, all_labels, acc = ft.eval(False)
    utils.confussion_matrix(all_labels, all_predictions, list(dataloaders['test'].dataset.classes),F1=False)
    return acc

acc = baseline(dataloaders, '')
print(f"Accuracy of baseline is {acc:.2f} %")

  0%|          | 0/133 [00:00<?, ?it/s]

texts ['An image of clothing with name: Shoes, and description: Cardigan in a soft fine knit with dropped shoulders, long sleeves, a visible seam at the back and no buttons.', 'An image of clothing with name: Jersey Basic, and description: Cardigan in a soft fine knit with dropped shoulders, long sleeves, a visible seam at the back and no buttons.', 'An image of clothing with name: Swimwear, and description: Cardigan in a soft fine knit with dropped shoulders, long sleeves, a visible seam at the back and no buttons.', 'An image of clothing with name: Dresses Ladies, and description: Cardigan in a soft fine knit with dropped shoulders, long sleeves, a visible seam at the back and no buttons.', 'An image of clothing with name: Blouses, and description: Cardigan in a soft fine knit with dropped shoulders, long sleeves, a visible seam at the back and no buttons.', 'An image of clothing with name: Dressed, and description: Cardigan in a soft fine knit with dropped shoulders, long sleeves, a

  1%|          | 1/133 [00:12<27:56, 12.70s/it]

texts ['An image of clothing with name: Shoes, and description: Small shoulder bag in cotton twill covered in plastic beads with a zip at the top, detachable metal chain shoulder strap and a zipped inner compartment. Cotton twill lining. Height 14 cm. Width 20 cm.', 'An image of clothing with name: Jersey Basic, and description: Small shoulder bag in cotton twill covered in plastic beads with a zip at the top, detachable metal chain shoulder strap and a zipped inner compartment. Cotton twill lining. Height 14 cm. Width 20 cm.', 'An image of clothing with name: Swimwear, and description: Small shoulder bag in cotton twill covered in plastic beads with a zip at the top, detachable metal chain shoulder strap and a zipped inner compartment. Cotton twill lining. Height 14 cm. Width 20 cm.', 'An image of clothing with name: Dresses Ladies, and description: Small shoulder bag in cotton twill covered in plastic beads with a zip at the top, detachable metal chain shoulder strap and a zipped inn

  2%|▏         | 2/133 [00:18<19:02,  8.72s/it]

...

texts ['An image of clothing with name: Shoes, and description: Set with a vest top and shorts in soft cotton jersey. Vest top with a decorative knot detail on each shoulder strap and a print motif. Shorts with side pockets and covered elastication at the waist.', 'An image of clothing with name: Jersey Basic, and description: Set with a vest top and shorts in soft cotton jersey. Vest top with a decorative knot detail on each shoulder strap and a print motif. Shorts with side pockets and covered elastication at the waist.', 'An image of clothing with name: Swimwear, and description: Set with a vest top and shorts in soft cotton jersey. Vest top with a decorative knot detail on each shoulder strap and a print motif. Shorts with side pockets and covered elastication at the waist.', 'An image of clothing with name: Dresses Ladies, and description: Set with a vest top and shorts in soft cotton jersey. Vest top with a decorative knot detail on each shoulder strap and a print motif. Shorts w

 21%|██        | 28/133 [02:04<07:15,  4.15s/it]

texts ['An image of clothing with name: Shoes, and description: Set with an Alice band and tail in faux fur. Alice band with appliqués in the shape of ears on top. Padded tail with ribbons and a hook and loop fastening. Length of tail approx. 35 cm.', 'An image of clothing with name: Jersey Basic, and description: Set with an Alice band and tail in faux fur. Alice band with appliqués in the shape of ears on top. Padded tail with ribbons and a hook and loop fastening. Length of tail approx. 35 cm.', 'An image of clothing with name: Swimwear, and description: Set with an Alice band and tail in faux fur. Alice band with appliqués in the shape of ears on top. Padded tail with ribbons and a hook and loop fastening. Length of tail approx. 35 cm.', 'An image of clothing with name: Dresses Ladies, and description: Set with an Alice band and tail in faux fur. Alice band with appliqués in the shape of ears on top. Padded tail with ribbons and a hook and loop fastening. Length of tail approx. 35 

 21%|██        | 28/133 [02:07<07:56,  4.54s/it]


KeyboardInterrupt: 